In [41]:
from openai import OpenAI
import os
import json
from datetime import datetime
import toml

#### Setup API Key

In [45]:
my_api_key = toml.load("config.toml")["openai"]["api_key"]
client = OpenAI(api_key = my_api_key)

#### Agent Prompt Template

In [46]:
agent_prompt = """
You are a professional task planning agent.

Your job is to breakdown the user's goal into a set of 5-10 clear, actionable steps. 
Make the steps concise, specific, and logical. Each steup should build on the previous one. 
For each step, include: Step description, Estimated time, Suggested tools or resources (if any)
The goal is: {goal}

Give your output as a numbered list.
"""

#### Agent Function

In [47]:
def breakdown_goal(goal):
    """
    # role = system: Tells GPT what kind of agent it is. This shapes its personality and rules.
    # role = user: The actual prompt (instructions + user's goal)
    # temperature: Controls randomness
        0.0 = deterministic, good for strict tasks.
        0.7 = balanced creativity and structure
    """
    response = client.chat.completions.create(
        model = "gpt-4o-mini",
        messages = [
            {"role": "system", "content": "You are a helpful assistant that breaks goals into tasks."},
            {"role": "user", "content": agent_prompt.format(goal=goal)}
        ],
        temperature = 0.7
    )
    return response.choices[0].message.content.strip()

#### Run the Agent

In [48]:
user_goal = "Plan a deeply researched, elegantly and concisely writter LinkedIn blogpost conforming to my writing style"
plan = breakdown_goal(user_goal)

print("Task Breakdown:\n", plan)

Task Breakdown:
 Here's a breakdown of the goal to plan a deeply researched, elegantly and concisely written LinkedIn blog post:

1. **Define the Blog Post Topic and Purpose**
   - **Step Description:** Choose a specific topic that aligns with your expertise and audience interests. Determine the purpose of the post (e.g., to inform, persuade, or share insights).
   - **Estimated Time:** 1 hour
   - **Suggested Tools/Resources:** Mind mapping tools (e.g., MindMeister), keyword research tools (e.g., Google Trends)

2. **Conduct Preliminary Research**
   - **Step Description:** Gather information on the chosen topic, including statistics, recent trends, and relevant articles. Take notes on key points that resonate with your writing style.
   - **Estimated Time:** 2-3 hours
   - **Suggested Tools/Resources:** Academic databases (e.g., Google Scholar), online articles, and industry reports

3. **Create an Outline for the Blog Post**
   - **Step Description:** Organize your research into a s

#### Add Simple Memory (JSON file)

- Save goals + tasks to a local JSON file
- Retreive previous sessions
- View task history

In [49]:
# Json file to save goals and task plan
MEMORY_FILE = "task_memory.json"

In [50]:
# Save task plan
def save_task(goal, task_list):

    # Open the memory file if it exists, else create one
    if os.path.exists(MEMORY_FILE):
        with open(MEMORY_FILE, "r") as f:
            memory = json.load(f)
    else:
        memory = []

    # Add to memory
    memory.append(
        {
            "timestamp": datetime.now().isoformat(),
            "goal": goal,
            "tasks": task_list
        }
    )

    # Update the memory in json file
    with open(MEMORY_FILE, "w") as f:
        json.dump(memory, f, indent=2)

In [51]:
# View task history
def view_memory():
    if not os.path.exists(MEMORY_FILE):
        return []
    with open(MEMORY_FILE, "r") as f:
        return json.load(f)

#### Updated Agent Function with memory

In [52]:
def breakdown_goal_with_memory(goal):
    """
    # role = system: Tells GPT what kind of agent it is. This shapes its personality and rules.
    # role = user: The actual prompt (instructions + user's goal)
    # temperature: Controls randomness
        0.0 = deterministic, good for strict tasks.
        0.7 = balanced creativity and structure
    """
    response = client.chat.completions.create(
        model = "gpt-4o-mini",
        messages = [
            {"role": "system", "content": "You are a helpful assistant that breaks goals into tasks."},
            {"role": "user", "content": agent_prompt.format(goal=goal)}
        ],
        temperature = 0.7
    )

    task_list = response.choices[0].message.content.strip()
    save_task(goal, task_list)
    
    return task_list

#### Run the updated Agent

In [53]:
user_goal = "Write an MBA assignment"
plan = breakdown_goal_with_memory(user_goal)

print("Task Breakdown:\n", plan)

Task Breakdown:
 Here’s a breakdown of your goal to write an MBA assignment into actionable steps:

1. **Understand Assignment Requirements**  
   - **Step Description**: Review the assignment prompt carefully to identify key requirements, objectives, and grading criteria.  
   - **Estimated Time**: 1 hour  
   - **Suggested Tools/Resources**: Assignment guidelines document, university rubric.

2. **Choose a Topic**  
   - **Step Description**: Select a relevant topic that aligns with the assignment requirements and personal interest.  
   - **Estimated Time**: 30 minutes  
   - **Suggested Tools/Resources**: Brainstorming tools (mind maps), academic articles for inspiration.

3. **Conduct Preliminary Research**  
   - **Step Description**: Gather initial information on the chosen topic through scholarly articles, books, and online resources.  
   - **Estimated Time**: 2 hours  
   - **Suggested Tools/Resources**: Google Scholar, university library databases, citation management softwa

#### View the memory

In [54]:
print("Viewing memory")

for entry in view_memory():
    print(f"\nGoal: {entry['goal']}")
    print(entry["tasks"])

Viewing memory

Goal: Write an MBA assignment
Sure! Here’s a breakdown of the goal "Write an MBA assignment" into actionable steps:

1. **Understand the Assignment Requirements**
   - **Step Description:** Read the assignment prompt carefully to grasp the objectives, format, and evaluation criteria.
   - **Estimated Time:** 1 hour
   - **Suggested Tools/Resources:** Course syllabus, assignment rubric, and any provided examples.

2. **Conduct Preliminary Research**
   - **Step Description:** Gather relevant materials and sources related to the assignment topic to build foundational knowledge.
   - **Estimated Time:** 2-3 hours
   - **Suggested Tools/Resources:** Academic databases (e.g., JSTOR, Google Scholar), library resources, and textbooks.

3. **Outline the Assignment Structure**
   - **Step Description:** Create a detailed outline that includes the introduction, main points, and conclusion to organize your thoughts.
   - **Estimated Time:** 1 hour
   - **Suggested Tools/Resources: